In [ ]:
!pip install yfinance plotly openai pandas numpy requests beautifulsoup4 lxml streamlit pyngrok --quiet


In [ ]:
%%writefile rebalance_backend.py
import os, re, time, requests, pandas as pd, numpy as np, yfinance as yf
import plotly.express as px
from bs4 import BeautifulSoup
from io import StringIO


# 1. Fetch S&P 500 tickers

def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {"User-Agent": "Mozilla/5.0"}
    html = requests.get(url, headers=headers).text
    df = pd.read_html(StringIO(html))[0]
    tickers = df["Symbol"].str.replace('.', '-', regex=False).tolist()
    print(f"Loaded {len(tickers)} tickers.")
    return tickers


# 2. Download Yahoo Finance data + Sharpe ratio

def get_stock_data(tickers, period="6mo"):
    data = {}
    for t in tickers:
        try:
            df = yf.download(t, period=period, progress=False, auto_adjust=True)
            if not df.empty:
                df["Return"] = df["Close"].pct_change()
                df["Volatility"] = df["Return"].rolling(20).std()
                data[t] = df.dropna()
        except Exception:
            pass
    print(f"Downloaded data for {len(data)} tickers.")
    return data

def compute_metrics(data):
    rows = []
    for t, df in data.items():
        if "Return" in df.columns and "Volatility" in df.columns:
            mean_ret = df["Return"].mean()
            vol = df["Volatility"].mean()
            sharpe = mean_ret / vol if vol else 0
            rows.append({"Ticker": t, "Sharpe": round(sharpe, 3)})
    return pd.DataFrame(rows)


# 3. Google News + GPT Sentiment

def fetch_recent_headlines(ticker, n=5):
    url = f"https://news.google.com/rss/search?q={ticker}+stock"
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=8)
        soup = BeautifulSoup(r.content, "xml")
        return [item.title.text for item in soup.find_all("item")[:n]]
    except Exception:
        return []

def analyze_sentiment(client, ticker, headlines):
    if not headlines:
        return 0.0
    prompt = (
        f"You are a financial analyst. Rate the overall sentiment toward {ticker} "
        f"from these headlines between -1 (very bearish) and +1 (very bullish). "
        "Respond with only one number.\n\n" + "\n".join(f"- {h}" for h in headlines)
    )
    try:
        r = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
        )
        out = r.choices[0].message.content.strip()
        m = re.search(r"-?\d+(\.\d+)?", out)
        return float(m.group()) if m else 0.0
    except Exception:
        return 0.0

def compute_sentiment_scores(client, tickers, top_n=50):
    scores = []
    for t in tickers[:top_n]:
        headlines = fetch_recent_headlines(t)
        val = analyze_sentiment(client, t, headlines)
        scores.append({"Ticker": t, "Sentiment": round(val, 3)})
        time.sleep(0.2)
    for t in tickers[top_n:]:
        scores.append({"Ticker": t, "Sentiment": 0.0})
    print(f"Generated sentiment for {top_n} tickers.")
    return pd.DataFrame(scores)


# 4. Portfolio generation + visualization

def generate_portfolio(tickers, client):
    data = get_stock_data(tickers)
    metrics_df = compute_metrics(data)
    sentiment_df = compute_sentiment_scores(client, tickers, top_n=len(tickers))

    df = pd.merge(metrics_df, sentiment_df, on="Ticker", how="inner").fillna(0)
    df["Score"] = 0.6 * df["Sharpe"] + 0.4 * df["Sentiment"]
    df["Weight"] = df["Score"] / df["Score"].sum()
    df["Weight_abs"] = df["Weight"].abs()

    # Pie chart – Top 25
    fig1 = px.pie(df.head(25), values="Weight_abs", names="Ticker",
                  title="Portfolio Allocation (Top 25 Weights)",
                  color_discrete_sequence=px.colors.qualitative.Set3)
    fig1.update_traces(textposition="inside", textinfo="percent+label")

    # Scatter – Sharpe vs Sentiment
    fig2 = px.scatter(df, x="Sharpe", y="Sentiment", size="Weight_abs",
                      color="Score", hover_name="Ticker",
                      title="Sharpe vs Sentiment (S&P 500 Universe)",
                      color_continuous_scale="Viridis")
    fig2.update_layout(yaxis_title="Sentiment", xaxis_title="Sharpe Ratio")

    return df.sort_values(by="Weight", ascending=False), fig1, fig2


Overwriting rebalance_backend.py


In [ ]:
%%writefile app.py
import streamlit as st
from openai import OpenAI
from rebalance_backend import get_sp500_tickers, generate_portfolio


# APP CONFIG

st.set_page_config(page_title="RebalanceAI Copilot", layout="wide")
st.title("RebalanceAI — S&P 500 Portfolio Copilot")

# Sidebar Info
st.sidebar.markdown("### Applied Generative AI Hackathon")
st.sidebar.markdown("""
**Group 4**
- Nishchay Linge Gowda
- Shivakumar Hassan Lokesh
- Supriya Singh
""")


# OPENAI SETUP

API_KEY = "API_KEY"
client = OpenAI(api_key=API_KEY)

# Model selection dropdown
model_options = {
    "GPT-4o-mini (fast)": "gpt-4o-mini",
    "GPT-4-Turbo (advanced)": "gpt-4-turbo",
    "GPT-3.5-Turbo (classic)": "gpt-3.5-turbo"
}
selected_model = st.sidebar.selectbox("🤖 Choose LLM Model:", list(model_options.keys()))
selected_model_name = model_options[selected_model]

# Session state
if "portfolio_df" not in st.session_state:
    st.session_state["portfolio_df"] = None
if "portfolio_fig1" not in st.session_state:
    st.session_state["portfolio_fig1"] = None
if "portfolio_fig2" not in st.session_state:
    st.session_state["portfolio_fig2"] = None


# PORTFOLIO ANALYSIS SECTION

st.subheader("Generate Your Portfolio")

num = st.slider("Select number of S&P 500 stocks to analyze", 10, 500, 50)
run = st.button("Generate Portfolio")

if run:
    st.info(f"⏳ Fetching market data and sentiment using {selected_model}…")
    tickers = get_sp500_tickers()[:num]
    # Pass selected model to backend
    st.session_state["portfolio_df"], st.session_state["portfolio_fig1"], st.session_state["portfolio_fig2"] = generate_portfolio(tickers, client)
    st.success(f"Portfolio generated successfully for {num} tickers!")

# Show visualizations and data table
if st.session_state["portfolio_df"] is not None:
    st.plotly_chart(st.session_state["portfolio_fig1"], use_container_width=True)
    st.plotly_chart(st.session_state["portfolio_fig2"], use_container_width=True)

    total_tickers = len(st.session_state["portfolio_df"])
    default_view = 25 if total_tickers >= 25 else total_tickers
    display_n = st.number_input("View Top N Tickers in Table", min_value=1, max_value=total_tickers, value=default_view, step=5)
    st.dataframe(st.session_state["portfolio_df"][["Ticker", "Sharpe", "Sentiment", "Score", "Weight"]].head(int(display_n)))


# COPILOT CHAT SECTION

st.markdown("---")
st.subheader("🤖 Copilot Chat — Ask About Your Portfolio")

query = st.text_input("Ask RebalanceAI (e.g. 'Top 5 bullish stocks' or 'Compare NVDA vs TSLA')")

if query:
    st.info(f"Analyzing with {selected_model}…")
    df = st.session_state["portfolio_df"]
    if df is None:
        context = "No portfolio data yet. Please run the analysis above first."
    else:
        context = df.to_string(index=False)

    prompt = (
        "You are a financial copilot analyzing the user’s portfolio. "
        "Base your answer on the data below.\n\n"
        f"{context}\n\nUser Query: {query}"
    )

    r = client.chat.completions.create(
        model=selected_model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )

    st.markdown("**🧭 Copilot Response:**")
    st.write(r.choices[0].message.content)


Overwriting app.py


In [ ]:
# Install & import
!pip install streamlit pyngrok --quiet
from pyngrok import ngrok
import threading, time, os

# Kill any old tunnels
ngrok.kill()

# Start Streamlit server in a thread
def start_streamlit():
    os.system("python -m streamlit run app.py --server.port 8501")

t = threading.Thread(target=start_streamlit, daemon=True)
t.start()

# Wait for Streamlit to boot up
time.sleep(8)

# Create public tunnel
public_url = ngrok.connect(8501)
print("🌍  Your public app URL:", public_url)


🌍  Your public app URL: NgrokTunnel: "https://semicylindrical-korbin-unperilously.ngrok-free.dev" -> "http://localhost:8501"
